In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### IMPORTS

In [2]:
# feature engineering libs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

# sklearn 
import sklearn
from sklearn.model_selection import train_test_split

# tensorflow related libs
import tensorflow as tf
from tensorflow.keras.layers import IntegerLookup, Normalization, StringLookup 

In [3]:
pd.set_option('display.max_rows', 500)

### HELPER FUNCTIONS

In [4]:
def create_model_checkpoint(model_name, save_path="model_experiment"):
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(save_path, model_name), 
        verbose=1, 
        save_best_only=True,
        monitor='val_accuracy',
    )

In [5]:
def dataframe_to_dataset (dataframe, is_test):
    data_frame = dataframe.copy()
    if is_test:
        ds = tf.data.Dataset.from_tensors(dict(data_frame))
    else:    
        labels = data_frame.pop("Survived")
        ds = tf.data.Dataset.from_tensor_slices((dict(data_frame), labels))
        ds = ds.shuffle(buffer_size=len(data_frame))
    return ds

In [6]:
def encode_numerical_feature (feature, name, dataset):
    normalizer = Normalization()
    
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))
    
    normalizer.adapt(feature_ds)
    encoded_feature = normalizer(feature)
    return encoded_feature

In [7]:
def encode_categorical_feature (feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    lookup = lookup_class(output_mode="binary")
    
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))
    
    lookup.adapt(feature_ds)
    encoded_feature = lookup(feature)
    return encoded_feature

In [8]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

### MANAGING DATA

In [9]:
# Imports dataset from Kaggle
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
len(train_df), len(test_df), train_df.shape, test_df.shape

(891, 418, (891, 12), (418, 11))

In [10]:
print(train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### CLEANING DATA

In [11]:
print(train_df.isna().sum())
print(test_df.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [12]:
# Filling age column missing values
train_df["Age"] = train_df.groupby(["Sex", "Pclass"])["Age"].apply(lambda x: x.fillna(x.median()))
test_df["Age"] = test_df.groupby(["Sex", "Pclass"])["Age"].apply(lambda x: x.fillna(x.median()))

# Filling embarked column missing values
train_df["Embarked"] = train_df["Embarked"].fillna('S')
test_df["Embarked"] = test_df["Embarked"].fillna('S')

# Filling fare column missing values
median_fare_value = train_df.groupby(["Pclass", "SibSp", "Parch"])["Fare"].median()[3][0][0]
train_df["Fare"] = train_df["Fare"].fillna(median_fare_value)
test_df["Fare"] = test_df["Fare"].fillna(median_fare_value)

# Creating deck column based on cabin column
train_df["Deck"] = train_df["Cabin"].apply(lambda s: s[0] if pd.notnull(s) else 'M')
test_df["Deck"] = test_df["Cabin"].apply(lambda s: s[0] if pd.notnull(s) else 'M')

train_df["Deck"] = train_df["Deck"].replace(['A','B','C', 'T'], 'ABC')
train_df["Deck"] = train_df["Deck"].replace(['D','E'], 'DE')
train_df["Deck"] = train_df["Deck"].replace(['F','G'], 'FG')
 
test_df["Deck"] = test_df["Deck"].replace(['A','B','C', 'T'], 'ABC')
test_df["Deck"] = test_df["Deck"].replace(['D','E'], 'DE')
test_df["Deck"] = test_df["Deck"].replace(['F','G'], 'FG')

In [13]:
# Binning continous features
train_df["Fare"] = pd.qcut(train_df["Fare"], 13)
test_df["Fare"] = pd.qcut(test_df["Fare"], 13)

train_df["Age"] = pd.qcut(train_df["Age"], 10)
test_df["Age"] = pd.qcut(test_df["Age"], 10)

In [14]:
# Frequency Encoding
train_df["Family_Size"] = train_df["SibSp"] + train_df["Parch"] + 1
test_df["Family_Size"] = test_df["SibSp"] + test_df["Parch"] + 1

train_df["Ticket_Frequency"] = train_df.groupby("Ticket")["Ticket"].transform('count')
test_df["Ticket_Frequency"] = test_df.groupby("Ticket")["Ticket"].transform('count')



In [20]:
# Title & IsMaried
train_df['Title'] = train_df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
train_df['Is_Maried'] = np.where(train_df["Title"] == 'Mrs', 1,0)

test_df["Title"] = test_df["Name"].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
test_df["Is_Maried"] = np.where(test_df["Title"] == 'Mrs', 1,0)

train_df["Title"] = train_df["Title"].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'], 'Miss/Mrs/Ms')
train_df["Title"] = train_df["Title"].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'],'Dr/Military/Noble/Clergy')
test_df["Title"] = test_df["Title"].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'],'Miss/Mrs/Ms')
test_df["Title"] = test_df["Title"].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'],'Dr/Military/Noble/Clergy')